In [1]:
import sys

# This to get the peerless target star DataFrame for example purposes
sys.path.append('/u/tdm/repositories/peerless/prediction')
sys.path.append('/u/tdm/repositories/peerless')
from targets import targets

# The action is here. Depends on vespa & isochrones.
from exosyspop.populations import KeplerBinaryPopulation

/u/tdm/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [2]:
pop = KeplerBinaryPopulation(targets, fB=0.4)

In [3]:
# Accessing secondary properties will initialize a secondary simulation,
# calling pop._generate_binaries().  The first time this is called, the
# secondary property regressors get trained.
pop.radius_B

flux_ratio regressor trained, R2=0.997480594694
q regressor trained, R2=0.999687889712


array([        nan,  0.94406818,         nan, ...,  0.45046734,
        0.7588677 ,  0.4555272 ])

In [4]:
# subsequent calls are much faster; e.g.
pop._generate_binaries()
print(pop.radius_B)
%timeit pop._generate_binaries()

[ 0.70221905         nan  0.42621156 ...,  0.54221126  0.89138903
         nan]
10 loops, best of 3: 81.1 ms per loop


In [5]:
# Similarly, accessing orbital properties will generate them
pop.period

array([  1.97281847e+02,   5.94887449e+05,   9.94727375e+04, ...,
         9.63095121e+02,   9.58370882e+02,   1.67927627e+01])

# Synthetic observations

In [6]:
# Now, we can observe and see what we see.  This takes into account
# duty cycle & data span, as well as geometry.
obs = pop.observe()
print(len(obs))
print(obs.columns)
obs.head()

537
Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec'],
      dtype='object')


,index,period,ecc,w,inc,a,aR,b_pri,b_sec,k,...,d_sec,T14_pri,T14_sec,T23_pri,T23_sec,dataspan,dutycycle,flux_ratio,n_pri,n_sec
0,13,297.310135,0.179304,3.409015,1.565552,1.623692e+13,206.936317,1.102682,1.002918,0.971737,...,0.191282,0.772010,0.729244,0.000000,0.000000,1459.789,0.6988,0.922287,3,4
1,114,1.775185,0.823747,2.551272,0.980192,4.407783e+11,7.087932,0.869877,2.343082,0.317603,...,0.000000,0.037099,0.000000,0.000000,0.000000,1459.789,0.6988,0.007475,571,0
2,162,3.689386,0.000000,0.226756,1.562825,7.501699e+11,10.479333,0.083534,0.083534,0.218700,...,0.001864,0.136564,0.136564,0.087137,0.087137,1459.789,0.8751,0.001868,345,342
3,546,514.292155,0.936780,0.817069,1.566063,2.052472e+13,385.401207,0.132708,0.704690,0.567181,...,0.030251,0.137905,0.656439,0.036382,0.000000,1459.789,0.6988,0.039709,2,3
4,771,133.342726,0.964515,1.466433,1.281343,8.601748e+12,172.414838,1.750950,84.222331,0.850389,...,0.000000,0.020714,0.000000,0.000000,0.000000,1459.789,0.6989,0.332515,6,0


In [7]:
# This is pretty fast, even when generating a new population each time:
%timeit pop.observe(new=True)

10 loops, best of 3: 164 ms per loop


In [8]:
# Even faster if we only generate new orbits.
%timeit pop.observe(new_orbits=True)

10 loops, best of 3: 82 ms per loop


In [11]:
# So we can predict the expected number of observations pretty easily.
import numpy as np
n_obs = np.array([len(pop.observe(new_orbits=True)) for i in range(100)])
n_obs.mean(), n_obs.std()

(533.49000000000001, 20.071121044924222)

In [12]:
# Notice that the above does not yet have trapezoidal parameters.  There are two options to generate these.
# Either we can set the fit_trap parameter, as follows:
obs = pop.observe(fit_trap=True)
print(len(obs))
obs.columns

534


Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'trap_dur_pri', u'trap_depth_pri',
       u'trap_slope_pri', u'trap_dur_sec', u'trap_depth_sec',
       u'trap_slope_sec'],
      dtype='object')

In [13]:
# All things considered, this is still pretty fast if we just need to do it a few times:
%timeit pop.observe(fit_trap=True)

1 loops, best of 3: 2.49 s per loop


In [14]:
# However, this is pretty slow if we want to do inference.  To help with this, we can 
# tell it to train & use a regression.  Training only happens once; by default with 10,000 
# synthetic observations.  This takes a minute or so.
obs = pop.observe(regr_trap=True)
print(len(obs))
obs.columns

Depth trained: R2=0.99821227196
Duration trained: R2=0.996635115395
Slope trained: R2=0.997047978794
529


Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'trap_dur_pri_regr',
       u'trap_depth_pri_regr', u'trap_slope_pri_regr', u'trap_dur_sec_regr',
       u'trap_depth_sec_regr', u'trap_slope_sec_regr'],
      dtype='object')

In [15]:
# Subsequent calls are much faster
%timeit pop.observe(regr_trap=True)

10 loops, best of 3: 33.8 ms per loop


In [14]:
# Even generating a new stellar population & observing it is pretty quick
%timeit pop.observe(regr_trap=True, new=True)

10 loops, best of 3: 181 ms per loop


In [16]:
# Or again, you can just generate new orbits (rather than new binaries & new orbits)
%timeit pop.observe(regr_trap=True, new_orbits=True)

10 loops, best of 3: 102 ms per loop


In [17]:
# Generating the training data used for the trapezoid shape regression above used
# this function, which can be otherwise useful to sample >N random observations 
# from the existing population.  `trap_regr` defaults to `True` here.  
# This function also takes `new` or `new_orbits` keywords.
obs_pop = pop.get_N_observed(N=10000, new_orbits=True)
print(len(obs_pop))
obs_pop.columns

10052


Index([u'index', u'period', u'ecc', u'w', u'inc', u'a', u'aR', u'b_pri',
       u'b_sec', u'k', u'tra', u'occ', u'd_pri', u'd_sec', u'T14_pri',
       u'T14_sec', u'T23_pri', u'T23_sec', u'dataspan', u'dutycycle',
       u'flux_ratio', u'n_pri', u'n_sec', u'trap_dur_pri_regr',
       u'trap_depth_pri_regr', u'trap_slope_pri_regr', u'trap_dur_sec_regr',
       u'trap_depth_sec_regr', u'trap_slope_sec_regr'],
      dtype='object')